In [3]:
%load_ext autoreload

%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from spikeA.Session import *
from spikeA.Spike_train_loader import Spike_train_loader
from spikeA.Cell_group import Cell_group
from spikeA.Animal_pose import Animal_pose
from spikeA.Session import Session
from spikeA.Dat_file_reader import Dat_file_reader
from spikeA.Intervals import Intervals
from spikeA.Spike_train import Spike_train
from spikeA.Spike_waveform import Spike_waveform
from tqdm import tqdm
import os.path
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mouse="bt0975"
date="28092021"
name=f"{mouse}-{date}-0105"

In [15]:
with open(f"/home/beate/Downloads/ses.pickle", 'rb') as loaded_ses_file:
    ses = pickle.load(loaded_ses_file)

In [11]:
ses.load_parameters_from_files()
print(ses)

<class 'spikeA.Session.Kilosort_session'>
name = bt0975-28092021-0105
path = /adata/electro/bt0975/bt0975-28092021-0105
subject = bt0975
session_dat_time = 2021-09-28 01:05:00
fileBase = /adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105
file_names = {'par': '/adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105.par', 'desen': '/adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105.desen', 'desel': '/adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105.desel', 'sampling_rate': '/adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105.sampling_rate_dat', 'px_per_cm': '/adata/electro/bt0975/bt0975-28092021-0105/bt0975-28092021-0105.px_per_cm', 'params': '/adata/electro/bt0975/bt0975-28092021-0105/params.py', 'amplitudes': '/adata/electro/bt0975/bt0975-28092021-0105/amplitudes.npy', 'channel_map': '/adata/electro/bt0975/bt0975-28092021-0105/channel_map.npy', 'channel_positions': '/adata/electro/bt0975/bt0975-28092021-0105/channel_positions.

In [12]:
stl = Spike_train_loader()
stl.load_spike_train_kilosort(ses)
cg = Cell_group(stl)

Number of good clusters: 37
